In [1]:
import torch.optim as optim
from torch_geometric.datasets import Planetoid
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import numpy as np
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.transforms import NormalizeFeatures
from torch_geometric.data import DataLoader

torch.set_printoptions(edgeitems=500)

# seed for reproducibility
seed = 5
torch.manual_seed(seed)
np.random.seed(seed)

### Data class info:
class 0: without autism associations
class 1: autism genes
class 2: 0.75 confidence
class 3: 0.5 confidence
class 4: unlabeled nodes

In [2]:
import read_data

data = read_data.read()

C:\Users\10455\OneDrive - Queen's University\Queens\CISC867\PANDA\src\read_data.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  autism_df['label'][autism_df['confidence'] == 0.5] = 0


In [3]:
data

Data(edge_index=[2, 811236], num_classes=23, test_mask=[23472], train_mask=[23472], x=[23472, 23472], y=[23472])

In [4]:
print(f'Contains isolated nodes: {data.contains_isolated_nodes()}')
print(f'Contains self-loops: {data.contains_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
# dataset = data
# data.train_mask = data.y >= 0

Contains isolated nodes: False
Contains self-loops: False
Is undirected: True
Average node degree: 34.56


In [5]:
data.train_mask.shape

torch.Size([23472])

In [6]:
data.train_mask.sum()

tensor(300)

In [7]:
data.test_mask.sum()
# data.test_mask = data.y >= 0

tensor(1535)

In [8]:
data.y[data.train_mask].shape


torch.Size([300])

In [9]:
data.y[data.test_mask].shape

torch.Size([1535])

In [10]:
# from torch_geometric.data import DataLoader
#
# loader = DataLoader(data, batch_size=32, shuffle=True)


In [11]:
# data_list = [data]
#

In [12]:
# dataset = DataLoader(data_list)
#

In [13]:
# dataset.num_node_features = data.num_node_features
# dataset.num_classes = data.num_classes


### Visualization Model Using Tensorboard Command
commandline run tensorboard
```
cd src
tensorboard --logdir log
```

In [14]:
# build model
from GCN import GCNStack

model = GCNStack(data.num_node_features, hidden_dim1=128, hidden_dim2=64, hidden_dim3=32, output_dim=data.num_classes, dropout=0.5)
print(model)


GCNStack(
  (convs): ModuleList(
    (0): GCNConv(23472, 128)
    (1): GCNConv(128, 64)
    (2): GCNConv(64, 32)
  )
  (lns): ModuleList(
    (0): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
    (1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
  )
  (post_mp): Sequential(
    (0): Linear(in_features=32, out_features=32, bias=True)
    (1): Linear(in_features=32, out_features=23, bias=True)
  )
)


In [15]:
# Running on GPU or CPU
use_GPU = True
device = torch.device('cuda' if torch.cuda.is_available() and use_GPU else 'cpu')
model, data = model.to(device), data.to(device)

In [16]:
device


device(type='cuda')

In [17]:
# torch.cuda.empty_cache()

In [18]:
def model_test(loader, model, is_validation=False, is_training=False):
    ''' Testing Code of the Model '''
    model.eval()

    correct = 0
    for data in loader:
        with torch.no_grad():
            emb, pred = model(data.x, data.edge_index)
            pred = pred.argmax(dim=1)
            label = data.y

        if is_training:
            mask = data.val_mask if is_validation else data.train_mask
        else: # testing
            mask = data.val_mask if is_validation else data.test_mask
        # node classification: only evaluate on nodes in test set
        pred = pred[mask]
        label = data.y[mask]
        # testing code
        # print(is_training, '$$pred', pred)
        # print(is_training, '%%label', label)
        correct += pred.eq(label).sum().item()
    total = 0
    for data in loader.dataset:
        if is_training:
            total += torch.sum(data.train_mask).item()
        else:
            total += torch.sum(data.test_mask).item()
    return correct / total

def model_train(dataset, writer, model, epoch_num, lr, weight_decay, momentum):
    ''' Training code of the model '''
    test_loader = loader = DataLoader(dataset, shuffle=False)

    # Optimizer
    # opt = optim.SGD(model.parameters(), lr=lr, weight_decay=weight_decay, momentum=momentum)
    opt = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

    # visualize the model architecture in tensorboard
    # writer.add_graph(model, ( data.x, data.edge_index ))

    # Training:
    for epoch in range(epoch_num + 1):
        total_loss = 0
        model.train()
        for batch in loader:
            #print(batch.train_mask, '----')
            opt.zero_grad()
            embedding, pred = model(batch.x, batch.edge_index)
            label = batch.y
            pred = pred[batch.train_mask]
            label = label[batch.train_mask]
            loss = model.loss(pred, label)
            loss.backward()
            opt.step()
            total_loss += loss.item() * batch.num_graphs
        total_loss /= len(loader.dataset)
        writer.add_scalar("loss", total_loss, epoch)

        # if epoch % 5 == 0:
        train_acc = model_test(test_loader, model, is_training=True)
        test_acc = model_test(test_loader, model, is_training=False)
        print("Epoch {}. Loss: {:.4f}. Train accuracy: {:.4f}. Test accuracy: {:.4f}".format(
            epoch, total_loss, train_acc, test_acc))
        writer.add_scalar("test accuracy", test_acc, epoch)

        if epoch % 20 == 0:
            name = 'epoch' + str(epoch)
            writer.add_embedding(embedding, global_step=epoch, tag=name, metadata=batch.y)

    return model

from datetime import datetime
from tensorboardX import SummaryWriter

writer = SummaryWriter("./log/" + datetime.now().strftime("%Y%m%d-%H%M%S"))

model_trained = model_train([data], writer, model, epoch_num=50, lr=0.0001, weight_decay=0.00001, momentum=0.9)

Epoch 0. Loss: 3.2817. Train accuracy: 0.4467. Test accuracy: 0.2678
Epoch 1. Loss: 3.0231. Train accuracy: 0.7167. Test accuracy: 0.6352
Epoch 2. Loss: 2.8353. Train accuracy: 0.7833. Test accuracy: 0.7179
Epoch 3. Loss: 2.7266. Train accuracy: 0.8167. Test accuracy: 0.7590
Epoch 4. Loss: 2.6686. Train accuracy: 0.8167. Test accuracy: 0.7792
Epoch 5. Loss: 2.6491. Train accuracy: 0.8267. Test accuracy: 0.7948
Epoch 6. Loss: 2.6068. Train accuracy: 0.8433. Test accuracy: 0.8065
Epoch 7. Loss: 2.5376. Train accuracy: 0.8467. Test accuracy: 0.8150
Epoch 8. Loss: 2.5379. Train accuracy: 0.8533. Test accuracy: 0.8235
Epoch 9. Loss: 2.4986. Train accuracy: 0.8533. Test accuracy: 0.8306
Epoch 10. Loss: 2.5066. Train accuracy: 0.8533. Test accuracy: 0.8352
Epoch 11. Loss: 2.5133. Train accuracy: 0.8533. Test accuracy: 0.8378
Epoch 12. Loss: 2.5238. Train accuracy: 0.8533. Test accuracy: 0.8404
Epoch 13. Loss: 2.4798. Train accuracy: 0.8533. Test accuracy: 0.8430
Epoch 14. Loss: 2.4355. Train 

In [19]:
test_acc = model_test( DataLoader([data], shuffle=False), model_trained, is_training=False)

In [20]:
test_acc

0.8690553745928339